In [41]:
import numpy as np
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

In [73]:
train_fn = '../out/memorization/head_val_scored.csv'
model_name = 'EleutherAI/gpt-j-6B'
gen_length = 20

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left')

In [59]:
train_df = pd.read_csv(train_fn, header=[0])
train_df.head(1)

,line,prefix,suffix,generated
0,0.0,Catalonia election: Puigdemont calls for talks...,'s biggest party. although pro-independence pa...,'s biggest party.\n\nThe Spanish government ha...


In [60]:
train_df = train_df.dropna()

In [80]:
def common_prefix(l1, l2):
    for i, item in enumerate(l1):
        if l2[i] != l1[i]:
            break
    return i

def overlap(l1, l2):
    s1, s2 = set(l1), set(l2)
    return s1.intersection(s2)

In [83]:
prefix_len, overlap_len = [], []
for i, row in train_df.iterrows():
    ground_tokenized = tokenizer(row['suffix'])['input_ids'][:gen_length]
    generated_tokenized = tokenizer(row['generated'])['input_ids']
    
    prefix_len.append(common_prefix(ground_tokenized, generated_tokenized))
    
    # you probably want to look at the overlap length as well
    overlap_len.append(len(overlap(ground_tokenized, generated_tokenized)))

In [ ]:
train_df['prefix_len'] = prefix_len
train_df['overlap_len'] = overlap_len